## 线性回归模型
#### 模型描述
<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mi>y</mi>
  <mo>=</mo>
  <mi>X</mi>
  <mo>&#x22C5;<!-- ⋅ --></mo>
  <mi>w</mi>
  <mo>+</mo>
  <mi>b</mi>
  <mo>+</mo>
  <mi>&#x03B7;<!-- η --></mi>
  <mo>,</mo>
  <mspace width="1em" />
  <mtext>for&#xA0;</mtext>
  <mi>&#x03B7;<!-- η --></mi>
  <mo>&#x223C;<!-- ∼ --></mo>
  <mrow class="MJX-TeXAtom-ORD">
    <mi class="MJX-tex-caligraphic" mathvariant="script">N</mi>
  </mrow>
  <mo stretchy="false">(</mo>
  <mn>0</mn>
  <mo>,</mo>
  <msup>
    <mi>&#x03C3;<!-- σ --></mi>
    <mn>2</mn>
  </msup>
  <mo stretchy="false">)</mo>
</math>

首先生成训练的数据,这里先生成一个Y 然后利用Y的shape去生成随机噪声，防止输入错误导致广播

In [2]:
from mxnet import ndarray as nd
from mxnet import autograd as ad

num_inputs = 2
num_examples = 1000

true_w = nd.array([2, -3.4])
true_b = 4.2
X=nd.random_normal(0,1,shape=(num_examples,num_inputs))
Y=nd.dot(X,true_w)+true_b
Y=Y+0.1*nd.random_normal(0,0.1,shape=Y.shape)

使用洗牌法，随机抽取训练的数据，一次的抽取量为bath_size，这里就代表以bath_size个样本做一次梯度下降训练。这里注意nd.take(array,index)用法

In [3]:
import random
bath_size=10
def data_iter():
    idx=list(range(num_examples))
    random.shuffle(idx)
    for i in range(0,num_examples,bath_size):
        j=nd.array(idx[i:min(i+bath_size,num_examples)])
        yield nd.take(X,j),nd.take(Y,j)

随机化初始值，并未每个参数生成导数空间。

In [4]:
w=nd.random_normal(0,1,shape=(2,1))
b = nd.zeros((1,))

params=[w,b]
for param in params:
    param.attach_grad()

定义模型：输出=function(输入)

In [5]:
def net(X):
    return nd.dot(X, w) + b

定义损失函数，使用最简单的平方根损失函数

In [6]:
def square_loss(yhat,y):
    return (yhat-y.reshape(yhat.shape))**2

定义优化函数，使用梯度下降算法

In [7]:
def SGD(params,lr):
    for param in params:
        param[:]=param-lr*param.grad

In [8]:
epoch=5
learn_rate=0.01
for e in range(epoch):
    total_loss=0
    num=0
    for data_X,data_Y in data_iter():
        with ad.record():
            yhat=net(data_X)
            loss=square_loss(yhat,data_Y)
        loss.backward()
        SGD(params,learn_rate)
        total_loss += nd.sum(loss).asscalar() 
        num+=1
    print(total_loss/num)

8.65511134212
0.00101087701536
0.00101263489138
0.00101048111435
0.00100880371261


In [9]:
print("params[0]:",params[0])
print("true_w:",true_w)
print("params[0]:",params[1])
print("true_w:",true_b)


params[0]: 
[[ 2.00021958]
 [-3.39915419]]
<NDArray 2x1 @cpu(0)>
true_w: 
[ 2.        -3.4000001]
<NDArray 2 @cpu(0)>
params[0]: 
[ 4.20022488]
<NDArray 1 @cpu(0)>
true_w: 4.2


### 使用Gluon

首先生成一组数据

In [3]:
from mxnet import ndarray as nd
from mxnet import autograd as ad
from mxnet import gluon
num_inputs = 2
num_examples = 1000

true_w = nd.array([2, -3.4])
true_b = 4.2
X=nd.random_normal(0,1,shape=(num_examples,num_inputs))
Y=nd.dot(X,true_w)+true_b
Y=Y+0.1*nd.random_normal(0,0.1,shape=Y.shape)

dataset = gluon.data.ArrayDataset(X, Y)：数据打包成一个训练集

data_iter = gluon.data.DataLoader(dataset, batch_size, shuffle=True):从训练集合中得到数据的生成器

In [19]:
batch_size = 10
dataset = gluon.data.ArrayDataset(X, Y)
data_iter = gluon.data.DataLoader(dataset, batch_size, shuffle=True)

使用gluon.nn.Sequential()函数返回一个网络，这个时候网络没有任何操作，这里网络类似于层的容器

In [20]:
net=gluon.nn.Sequential()

使用gluon.nn.Dense(1)函数返回一个层，这里输入参数的个数可以由网络在接受输入的时候或者上一层的信息推断，因此我们只要给出输出参数个数即可。

在使用gluon.nn.Dense()生成一层后，我们使用网络的add()方法添加进net这个网络中。

In [21]:
net.add(gluon.nn.Dense(1))

在构建好网络后（即向net这个容器中添加完毕层后），使用net的initialize()方法，这样可以初始化整个网络，这里包含了对导数的分配等等。

In [22]:
net.initialize()

In [23]:
loss_function=gluon.loss.L2Loss()

这里使用gluon.Trainer(参数集，优化方法，其他参数)定义一个训练器。我们借助这个训练器去训练网络的参数。

In [24]:
Trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.01})

其余都不变，这里需要注意的是训练的过程我们不在使用自己写的SGD函数手动训练,而是使用先前定义的训练器Trainer。Trainer.step(batch_size)会自动训练网络。

In [25]:
epochs=5
batch_size=10
for e in range(epochs):
    total_loss=0
    for data,label in data_iter:
        with ad.record():
            output=net(data)
            loss=loss_function(output,label)
        loss.backward()
        Trainer.step(batch_size)
        total_loss=nd.sum(loss).asscalar()
    print("Epoch %d, average loss: %f" % (e, total_loss/num_examples))    

Epoch 0, average loss: 0.023016
Epoch 1, average loss: 0.002710
Epoch 2, average loss: 0.000224
Epoch 3, average loss: 0.000018
Epoch 4, average loss: 0.000007
